# 医学生のための画像レジストレーション入門

担当教員：〇〇 〇〇

---

## 1. はじめに：なぜ画像レジストレーションが重要なのか？

皆さん、こんにちは。この講義では、**画像レジストレーション（Image Registration）**、日本語では「画像の位置合わせ」と呼ばれる技術について学んでいきます。

医学の世界では、CTやMRIなど、様々な種類の画像を扱います。例えば、一人の患者さんに対して、

- 異なる時期に撮影したCT画像（病変の経時的変化を見るため）
- CTとMRIの両方の画像（骨の詳細な情報はCT、軟部組織の情報はMRIで得るため）

などを比較・統合して診断や治療計画に役立てます。しかし、撮影時の患者さんの僅かな体の動きや呼吸による内臓の動きなどにより、これらの画像は微妙に、時には大きく位置がずれています。

この「位置のずれ」をコンピュータの力で補正し、複数の画像を正確に重ね合わせる技術が、画像レジストレーションです。

この技術により、
- **病変の正確な変化の追跡**
- **複数の画像情報の統合による診断能の向上**
- **手術ナビゲーションや放射線治療計画の精度向上**

などが可能になります。

この講義では、画像処理の専門家ではない皆さんにも理解できるよう、レジストレーションの基本的な考え方と代表的な手法を、実際のコードを動かしながら体験してもらいます。

## 2. 準備：ライブラリとデータの準備

In [ ]:
%pip install -q SimpleITK scikit-image matplotlib

import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from skimage import data
%matplotlib inline

def start_plot():
    global metric_values
    metric_values = []

def end_plot():
    global metric_values
    del metric_values
    clear_output(wait=True)

def plot_values(registration_method):
    global metric_values
    metric_values.append(registration_method.GetMetricValue())
    clear_output(wait=True)
    plt.plot(metric_values, 'r-')
    plt.xlabel('Iteration Number')
    plt.ylabel('Metric Value')
    plt.grid(True)
    plt.show()

def checkerboard_plot(image1, image2, pattern=[12,12]):
    checker_board_filter = sitk.CheckerBoardImageFilter()
    checker_board_filter.SetCheckerPattern(pattern)
    result = checker_board_filter.Execute(image1, image2)
    plt.imshow(sitk.GetArrayViewFromImage(result), cmap=plt.cm.Greys_r)
    plt.axis('off')
    plt.show()

print("ライブラリとユーティリティ関数の準備が完了しました。")

### テストデータの生成

In [ ]:
image_3d_np = data.brain()
slice_index = image_3d_np.shape[0] // 2
fixed_image_np = image_3d_np[slice_index, :, :].astype(np.float32)
fixed_image = sitk.GetImageFromArray(fixed_image_np)
fixed_image.SetSpacing([0.5, 0.5])

true_transform = sitk.Euler2DTransform()
center_point = np.array(fixed_image.GetSize())/2.0
physical_center = fixed_image.TransformContinuousIndexToPhysicalPoint(center_point)
true_transform.SetCenter(physical_center)
true_transform.SetAngle(np.deg2rad(5))
true_transform.SetTranslation((10.0, -5.0))

moving_image = sitk.Resample(fixed_image, fixed_image, true_transform, sitk.sitkLinear, 0.0, fixed_image.GetPixelID())

print("固定画像と移動画像の準備が完了しました。")

## 3. 位置合わせ前の状態を確認する

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(sitk.GetArrayViewFromImage(fixed_image), cmap=plt.cm.Greys_r)
plt.title('固定画像 (Fixed)')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(sitk.GetArrayViewFromImage(moving_image), cmap=plt.cm.Greys_r)
plt.title('移動画像 (Moving)')
plt.axis('off')
plt.show()

print("位置合わせ前のチェッカーボード表示：")
checkerboard_plot(fixed_image, moving_image)

## 4. レジストレーションの実践

### 手法1：Rigid Body Transform (剛体変換)
**【このセルを根本的に修正しました】**

In [ ]:
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)

# 最適化手法の設定
registration_method.SetOptimizerAsGradientDescent(learningRate=0.1, numberOfIterations=200, convergenceMinimumValue=1e-6, convergenceWindowSize=10)

# ★★★ 修正点：最適化スケールの手動設定 ★★★
# 回転(ラジアン単位)と移動(mm単位)ではスケールが全く違うため、手動でバランスを取ります。
# 回転のスケールを大きくすることで、回転の更新ステップを小さく、慎重に行うようにします。
# パラメータの順番は [角度, x移動量, y移動量]
registration_method.SetOptimizerScales([200.0, 1.0, 1.0])

initial_transform = sitk.CenteredTransformInitializer(fixed_image, moving_image, sitk.Euler2DTransform(), sitk.CenteredTransformInitializerFilter.GEOMETRY)
registration_method.SetInitialTransform(initial_transform)

registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))

print("剛体変換によるレジストレーションを開始します...")
rigid_transform = registration_method.Execute(fixed_image, moving_image)
print("完了しました。")

moving_resampled_rigid = sitk.Resample(moving_image, fixed_image, rigid_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
print("\n剛体変換後のチェッカーボード表示：")
checkerboard_plot(fixed_image, moving_resampled_rigid)

### 手法2：Affine Registration (アフィン変換)
**【このセルを修正しました】**

In [ ]:
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)

registration_method.SetOptimizerAsGradientDescent(learningRate=0.1, numberOfIterations=200, convergenceMinimumValue=1e-6, convergenceWindowSize=10)

# ★★★ 修正点：最適化スケールの手動設定 ★★★
# アフィン変換のパラメータも、回転・スケール成分と移動成分でスケールが異なります。
# パラメータ：[matrix(0,0), matrix(0,1), matrix(1,0), matrix(1,1), translation_x, translation_y]
scales = np.array([1.0, 1.0, 1.0, 1.0, 1.0/1000, 1.0/1000]) # 逆数を設定する流儀もあるが、今回は他と合わせる
scales = [1.0, 1.0, 1.0, 1.0, 200.0, 200.0] # こちらの方が直感的。移動成分の更新を慎重にする
registration_method.SetOptimizerScales([1, 1, 1, 1, 1.0/200, 1.0/200]) # SetOptimizerScalesFromPhysicalShiftの挙動に近い形

 # 最も安定するアプローチ: SetOptimizerScalesFromJacobian を使う
registration_method.SetOptimizerScalesFromJacobian() 

initial_transform = sitk.CenteredTransformInitializer(fixed_image, moving_image, sitk.AffineTransform(fixed_image.GetDimension()), sitk.CenteredTransformInitializerFilter.GEOMETRY)
registration_method.SetInitialTransform(initial_transform)

registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))

print("アフィン変換によるレジストレーションを開始します...")
affine_transform = registration_method.Execute(fixed_image, moving_image)
print("完了しました。")

moving_resampled_affine = sitk.Resample(moving_image, fixed_image, affine_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
print("\nアフィン変換後のチェッカーボード表示：")
checkerboard_plot(fixed_image, moving_resampled_affine)

### 手法3：Non-rigid Registration (非剛体変換)

In [ ]:
registration_method = sitk.ImageRegistrationMethod()
registration_method.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)

# 非剛体変換ではパラメータ数が非常に多いため、自動スケール設定が効果的です
registration_method.SetOptimizerAsGradientDescent(learningRate=0.1, numberOfIterations=200, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetOptimizerScalesFromJacobian()

transform_domain_mesh_size = [8] * fixed_image.GetDimension()
initial_bspline_transform = sitk.BSplineTransformInitializer(
    image1 = fixed_image, 
    transformDomainMeshSize = transform_domain_mesh_size
)

initial_transform = sitk.CompositeTransform([affine_transform, initial_bspline_transform])
registration_method.SetInitialTransform(initial_transform)

registration_method.AddCommand(sitk.sitkStartEvent, start_plot)
registration_method.AddCommand(sitk.sitkEndEvent, end_plot)
registration_method.AddCommand(sitk.sitkIterationEvent, lambda: plot_values(registration_method))

print("非剛体変換によるレジストレーションを開始します...")
nonrigid_transform = registration_method.Execute(fixed_image, moving_image)
print("完了しました。")

moving_resampled_nonrigid = sitk.Resample(moving_image, fixed_image, nonrigid_transform, sitk.sitkLinear, 0.0, moving_image.GetPixelID())
print("\n非剛体変換後のチェッカーボード表示：")
checkerboard_plot(fixed_image, moving_resampled_nonrigid)

## 5. 結果の比較とまとめ

In [ ]:
plt.figure(figsize=(18, 10))

# 1. 位置合わせ前
plt.subplot(2, 2, 1)
checker_board_before = sitk.CheckerBoardImageFilter()
checker_board_before.SetCheckerPattern([12,12])
plt.imshow(sitk.GetArrayViewFromImage(checker_board_before.Execute(fixed_image, moving_image)), cmap=plt.cm.Greys_r)
plt.title('1. 位置合わせ前', fontsize=16)
plt.axis('off')

# 2. 剛体変換後
plt.subplot(2, 2, 2)
checker_board_rigid = sitk.CheckerBoardImageFilter()
checker_board_rigid.SetCheckerPattern([12,12])
plt.imshow(sitk.GetArrayViewFromImage(checker_board_rigid.Execute(fixed_image, moving_resampled_rigid)), cmap=plt.cm.Greys_r)
plt.title('2. 剛体変換 (Rigid)', fontsize=16)
plt.axis('off')

# 3. アフィン変換後
plt.subplot(2, 2, 3)
checker_board_affine = sitk.CheckerBoardImageFilter()
checker_board_affine.SetCheckerPattern([12,12])
plt.imshow(sitk.GetArrayViewFromImage(checker_board_affine.Execute(fixed_image, moving_resampled_affine)), cmap=plt.cm.Greys_r)
plt.title('3. アフィン変換 (Affine)', fontsize=16)
plt.axis('off')

# 4. 非剛体変換後
plt.subplot(2, 2, 4)
checker_board_nonrigid = sitk.CheckerBoardImageFilter()
checker_board_nonrigid.SetCheckerPattern([12,12])
plt.imshow(sitk.GetArrayViewFromImage(checker_board_nonrigid.Execute(fixed_image, moving_resampled_nonrigid)), cmap=plt.cm.Greys_r)
plt.title('4. 非剛体変換 (Non-rigid)', fontsize=16)
plt.axis('off')

plt.tight_layout()
plt.show()

### まとめ